In [1]:
from datasets import load_dataset
docs = load_dataset(f"mhhmm/leetcode-solutions-python", split="train")

Found cached dataset json (C:/Users/Tommaso/.cache/huggingface/datasets/mhhmm___json/mhhmm--leetcode-solutions-python-c6d2758e3a6cc905/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


In [2]:
docs.column_names

['id', 'code_only', 'code_with_problem', 'code_with_data', 'explanation_only']

In [3]:
docs.shape

(2359, 5)

In [4]:
docs["code_only"][1]

'```python\nclass ListNode:\n    def __init__(self, val=0, next=None):\n        self.val = val\n        self.next = next\n\ndef addTwoNumbers(l1, l2):\n    dummy = ListNode(0)\n    current = dummy\n    carry = 0\n\n    while l1 or l2 or carry:\n        sum_val = (l1.val if l1 else 0) + (l2.val if l2 else 0) + carry\n        carry = sum_val // 10\n        current.next = ListNode(sum_val % 10)\n        current = current.next\n\n        if l1: l1 = l1.next\n        if l2: l2 = l2.next\n\n    return dummy.next\n```\n\n'

In [5]:
import libcst as cst
import re
import random
from openbugger.bugger import Bugger, bugger_example
from time import perf_counter

In [6]:
#import all bugs from openbugger
from openbugger.bugs.controlflow import ForgettingToUpdateVariableTransformer, InfiniteWhileTransformer, gen_OffByKIndexTransformer, IncorrectExceptionHandlerTransformer,MissingArgumentTransformer,ReturningEarlyTransformer
from openbugger.bugs.data import IncorrectVariableInitializationTransformer, VariableNameTypoTransformer, MutableDefaultArgumentTransformer, UseBeforeDefinitionTransformer
from openbugger.bugs.logical import gen_ComparisonTargetTransfomer, ComparisonSwapTransformer
from openbugger.bugs.type import IncorrectTypeTransformer, NonExistingMethodTransformer, SwapForTransformer
from openbugger.bugs.numpy import NumpyArrayCreationTransformer, NumpyMethodMisuseTransformer, NumpyReshapeMisuseTransformer, NumpyArangeMisuseTransformer, NumpyAxisMisuseTransformer

In [7]:
transformers = [gen_OffByKIndexTransformer(1)]
# Get the script as a string
script = "while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: \n  y = 1 + 2\nx[1:2]\nx[1]"
bugger = Bugger(transformers)
# Parse the script into a CST
module = cst.parse_module(script)

tainted = bugger.transformers[0].mutate(module)
tainted.code
# clean = bugger.apply(tainted,debug=True

'while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: \n  y = 1 + 2\nx[1:2]\nx[2]'

In [8]:
code_list = []
problem_list = []
data_list = []
cst_list = []
bad_code = []
explanation_list = []
for code_with_data,code_with_problem,code,explanation in zip(docs["code_with_data"],docs["code_with_problem"],docs["code_only"],docs["explanation_only"]):
    #remove ```python and ``` at the end 
    good_code= False
    clean_code = re.sub(r'```python', '', code)
    clean_code = re.sub(r'```.*', '', clean_code).strip()

    split_data = code_with_data.split('```python')
    split_problem = code_with_problem.split('```python')
    if len(split_data) > 1 and len(split_problem) > 1:
        data = split_data[0]
        problem = split_problem[0]
        
        try:
            cst_tree = cst.parse_module(clean_code)
            good_code = True
            code_list.append(clean_code)
            cst_list.append(cst_tree)
            data_list.append(data)
            problem_list.append(problem)
            explanation_list.append(explanation)
        except:
            bad_code.append(clean_code)

In [9]:
print(len(code_list),len(cst_list),len(data_list),len(problem_list),len(bad_code),len(explanation_list))

2348 2348 2348 2348 2 2348


In [10]:
#create a dictionary where keys are bugs type and values are the bug classes
bug_dict = {"controlflow":[ForgettingToUpdateVariableTransformer, InfiniteWhileTransformer, gen_OffByKIndexTransformer(),IncorrectExceptionHandlerTransformer,MissingArgumentTransformer,ReturningEarlyTransformer],
            "data":[IncorrectVariableInitializationTransformer, VariableNameTypoTransformer, MutableDefaultArgumentTransformer, UseBeforeDefinitionTransformer],
            "logical":[gen_ComparisonTargetTransfomer(), ComparisonSwapTransformer],
            "type":[IncorrectTypeTransformer, NonExistingMethodTransformer, SwapForTransformer],
            "numpy":[NumpyArrayCreationTransformer, NumpyMethodMisuseTransformer, NumpyReshapeMisuseTransformer, NumpyArangeMisuseTransformer, NumpyAxisMisuseTransformer]}
def sample_bug(bug_dict):
    # Select a random bug type
    bug_type = random.choice(list(bug_dict.keys()))
    # Select a random bug from the chosen type
    bug_class = random.choice(bug_dict[bug_type])
    return bug_type, bug_class

In [11]:

#sample a random from code_list and applies bugger_example
random_script = random.sample(code_list,1)[0]

In [12]:
tainted = bugger.transformers[0].mutate(module)

In [13]:
tainted.code

'while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: \n  y = 1 + 2\nx[1:2]\nx[1]'

In [14]:
script = """def add(a, b):
    MAX = 0x7FFFFFFF
    MASK = 0xFFFFFFFF
    while b:
        a, b = (a ^ b) & MASK, ((a & b) << 1) & MASK
    return a if a <= MAX else ~(a ^ MASK)"""

In [15]:
# print(random_bug[1])
# print(script)
bugger = Bugger([UseBeforeDefinitionTransformer])
# Parse the script into a CST
module = cst.parse_module(script)

tainted = bugger.transformers[0].mutate(module)
tainted = bugger.position_updater.transform_module(tainted)
clean = bugger.inverse.debug(tainted,bugger.transformers[0].id)
tainted.code

'def add(a, b):\n    ab = a + 1\n    MAX = 0x7FFFFFFF\n    MASK = 0xFFFFFFFF\n    while b:\n        a, b = (a ^ b) & MASK, ((a & b) << 1) & MASK\n    return a if a <= MAX else ~(a ^ MASK)'

In [16]:
# for script in code_list:
#     random_bug = UseBeforeDefinitionTransformer # sample_bug(bug_dict)
#     transformers=[random_bug]
#     bugger = Bugger(transformers)
#     # Parse the script into a CST
#     module = cst.parse_module(script)
#     tainted = bugger.apply(module)

#     clean = bugger.apply(tainted,debug=True)

In [17]:
# fail = 0
# fail_bugs = 0
# fail_inversions = 0
# bugs = 0
# inversions = 0
# for script in code_list:
#     try:
#         random_bug = ForgettingToUpdateVariableTransformer # sample_bug(bug_dict)
#         transformers=[random_bug]
#         bugger = Bugger(transformers)
#         # Parse the script into a CST
#         module = cst.parse_module(script)
#         tainted = bugger.apply(module)
#         if not module.deep_equals(tainted):
#             bugs+=1
#             try:
#                 clean = bugger.apply(tainted,debug=True)
#                 # bugger.print()
#                 if module.deep_equals(clean):
#                     inversions+=1
#             except:
#                 fail_inversions +=1
        
#     except:
#         fail+=1
#         print(fail)
        

In [18]:
# print("Bug Name: {name}, fail {fail}, fail_bugs {fail_bugs}, fail_inversions {fail_inversions}, bugs {bugs}, inversions {inversions}, total {total} ".format(name= random_bug.__name__,fail=fail,fail_bugs=fail_bugs,fail_inversions=fail_inversions,bugs=bugs,inversions=inversions, total=len(code_list)))

In [19]:
class BuggerAnalyzer:
    def __init__(self, bug_dict):
        self.bug_dict = bug_dict

    def apply_bugs(self, cst_list):
        results = {}

        # Iterate through each type of bug and associated transformers
        for bug_type, transformers in self.bug_dict.items():
            results[bug_type] = {}
            for transformer in transformers:
                # Initialize result dictionary for the transformer
                results[bug_type][transformer.__name__] = {
                    "successful_bugs": [],
                    "successful_inversions": [],
                    "failed_inversions": [],
                    "failed_transformations": [],
                    "time_to_bug": 0
                }
                # Measure bug application time
                start = perf_counter()
                
                # Apply bug to each module in the list
                for idx, module in enumerate(cst_list):
                    try:
                        bugger = Bugger([transformer])
                        tainted = bugger.apply(module)
                        # Check if the bug application was successful
                        if not module.deep_equals(tainted):
                            results[bug_type][transformer.__name__]["successful_bugs"].append(idx)
                            # Attempt to invert the transformation
                            try:
                                clean = bugger.apply(tainted, debug=True)
                                # Check if the inversion was successful
                                if module.deep_equals(clean):
                                    results[bug_type][transformer.__name__]["successful_inversions"].append(idx)
                                else:
                                    results[bug_type][transformer.__name__]["failed_inversions"].append(idx)
                            except:
                                results[bug_type][transformer.__name__]["failed_inversions"].append(idx)
                    except:
                        results[bug_type][transformer.__name__]["failed_transformations"].append(idx)
                end = perf_counter()
                results[bug_type][transformer.__name__]["time_to_bug"] = end-start
                print(f"Time to bug for {transformer.__name__}: ", end - start)

        return results


In [20]:
analyzer = BuggerAnalyzer(bug_dict=bug_dict)
results = analyzer.apply_bugs(cst_list)


Time to bug for ForgettingToUpdateVariableTransformer:  106.06652310001664
Time to bug for InfiniteWhileTransformer:  85.74230270000407
Time to bug for OffByKIndexTransformer:  107.97042100003455
Time to bug for IncorrectExceptionHandlerTransformer:  72.03556499996921
Time to bug for MissingArgumentTransformer:  104.13743010000326
Time to bug for ReturningEarlyTransformer:  107.5017256999854
Time to bug for IncorrectVariableInitializationTransformer:  103.38590380002279
Time to bug for VariableNameTypoTransformer:  107.76823859999422
Time to bug for MutableDefaultArgumentTransformer:  134.2774347999948
Time to bug for UseBeforeDefinitionTransformer:  117.8377967000124
Time to bug for ComparisonTargetTransformer:  77.02554569998756
Time to bug for ComparisonSwapTransformer:  103.71245380002074
Time to bug for IncorrectTypeTransformer:  107.01186560001224
Time to bug for NonExistingMethodTransformer:  84.15713040000992
Time to bug for SwapForTransformer:  100.6445459999959
Time to bug fo

In [21]:
import json
def save_results(results, file_path):
    with open(file_path, 'w') as file:
        json.dump(results, file, default=lambda o: o.__dict__)
save_results(results, 'bugger_analys_results.json')


In [26]:
def print_summary_statistics(results):
    for bug_type, transformers in results.items():
        print(f"\nBug Type: {bug_type}")
        for transformer, stats in transformers.items():
            print(f"\nTransformer: {transformer}")
            successful_bugs_count = len(stats["successful_bugs"])
            successful_inversions_count = len(stats["successful_inversions"])
            failed_inversions_count = len(stats["failed_inversions"])
            failed_transformations_count = len(stats["failed_transformations"])
            time_to_bug = stats["time_to_bug"]
            print(f"Successful Bugs: {successful_bugs_count}")
            print(f"Successful Inversions: {successful_inversions_count}")
            print(f"Failed Inversions: {failed_inversions_count}")
            print(f"Failed Transformations: {failed_transformations_count}")
            print(f"Time to Bug: {time_to_bug:.2f} seconds")


In [27]:
print_summary_statistics(results)



Bug Type: controlflow

Transformer: ForgettingToUpdateVariableTransformer
Successful Bugs: 2249
Successful Inversions: 2233
Failed Inversions: 16
Failed Transformations: 0
Time to Bug: 106.07 seconds

Transformer: InfiniteWhileTransformer
Successful Bugs: 654
Successful Inversions: 445
Failed Inversions: 209
Failed Transformations: 0
Time to Bug: 85.74 seconds

Transformer: OffByKIndexTransformer
Successful Bugs: 1626
Successful Inversions: 1618
Failed Inversions: 8
Failed Transformations: 0
Time to Bug: 107.97 seconds

Transformer: IncorrectExceptionHandlerTransformer
Successful Bugs: 2
Successful Inversions: 2
Failed Inversions: 0
Failed Transformations: 0
Time to Bug: 72.04 seconds

Transformer: MissingArgumentTransformer
Successful Bugs: 2161
Successful Inversions: 1899
Failed Inversions: 262
Failed Transformations: 0
Time to Bug: 104.14 seconds

Transformer: ReturningEarlyTransformer
Successful Bugs: 2325
Successful Inversions: 2319
Failed Inversions: 6
Failed Transformations: 0


In [28]:
def successful_inversions_dict(results):
    successful_inversions = {}

    for bug_type, transformers in results.items():
        for transformer, stats in transformers.items():
            for idx in stats["successful_inversions"]:
                if idx not in successful_inversions:
                    successful_inversions[idx] = set()
                successful_inversions[idx].add(transformer)

    return successful_inversions


In [29]:
successful_inversions = successful_inversions_dict(results)


In [31]:
for idx, transformers in successful_inversions.items():
    if idx < 10:
        print(f"Index {idx}:\n\tSuccessful Inversions: {transformers}")
    else:
        break

<class 'set'>
Index 0:
	Successful Inversions: {'UseBeforeDefinitionTransformer', 'ReturningEarlyTransformer', 'MissingArgumentTransformer', 'ForgettingToUpdateVariableTransformer', 'MutableDefaultArgumentTransformer', 'VariableNameTypoTransformer', 'OffByKIndexTransformer', 'ComparisonSwapTransformer', 'SwapForTransformer'}
<class 'set'>
Index 1:
	Successful Inversions: {'ReturningEarlyTransformer', 'MissingArgumentTransformer', 'IncorrectTypeTransformer', 'IncorrectVariableInitializationTransformer', 'ForgettingToUpdateVariableTransformer', 'VariableNameTypoTransformer'}
<class 'set'>
Index 2:
	Successful Inversions: {'ReturningEarlyTransformer', 'MissingArgumentTransformer', 'IncorrectTypeTransformer', 'NonExistingMethodTransformer', 'IncorrectVariableInitializationTransformer', 'ForgettingToUpdateVariableTransformer', 'MutableDefaultArgumentTransformer', 'VariableNameTypoTransformer', 'OffByKIndexTransformer', 'ComparisonSwapTransformer'}
<class 'set'>
Index 3:
	Successful Inversio